In [1]:
from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from config import OWM_API_key as key, port, host
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, InvalidDocument, DuplicateKeyError
# import json
import os
from os import path


API_key = key
host = host
port = port
filename = 'resources/zip_list.csv'
global zipcode
owm = OWM(API_key) # the OWM object
global obs
global reception_time
global zid
global zlon
global zlat
global client

In [2]:
def set_location(code):
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    global obs, zlat, zlon
    obs = owm.weather_at_zip_code(f'{code}', 'us')
    location = obs.get_location()
    zlon = location.get_lon()
    zlat = location.get_lat()
    return


def current():
    ''' Dump the current weather to a json

        :return current: the currently observed weather data
        :type current: dict
    '''
    global obs
    current = obs.to_JSON() # dump all that weather data to a json object
    return(current)


def five_day():
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code\
        :type five_day: dict
    '''
    global obs, zlat, zlon
    forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    forecast = forecaster.get_forecast()
    five_day = forecast.to_JSON()
    return(five_day)


def check_db_access(host, port):
    ''' A check that there is write access to the database
    
        :param host: the database host
        :type host: string
        :param port: the database connection port
        :type port: int
    '''
    client = MongoClient(host=host, port=port)
    try:
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print('Connection made')
    except ConnectionFailure:
        print("Server not available")
        return

    # check the database connections
        # Get a count of the number of databases at the connection (accessible through that port)
        # before attempting to add to it
    db_count_pre = len(client.list_database_names())
        # Add a database and collection
    db = client.test_db
    col = db.test_col

    # Insert something to the db
    post = {'name':'Chuck VanHoff',
           'age':'38',
           'hobby':'gardening'
           }
    col.insert_one(post)

        # Get a count of the databases after adding one
    db_count_post = len(client.list_database_names())

    if db_count_pre-db_count_post>=0:
        print('Your conneciton is flipped up')
    else:
        print('You have write access')

    client.drop_database(db)
    return(client)

def to_json(data, code):
    ''' Store the collected data as a json file in the case that the database
        is not connected or otherwise unavailable.
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param code: the zip code associated with the data from the list codes
        :type code: sting
    '''
    collection = data
    directory = os.getcwd()
    save_path = path.join(directory, 'Data', f'{code}.json')
    Data = open(save_path, 'a+')
    Data.write(collection)
    Data.close()
    return

def load(data, client):
    ''' Load the data to the database if possible, otherwise write to json file. 
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param client: the pymongo client object
        :type client: MongoClient
    '''
    if type(data) == dict:
        try:
            db = client.forcast
            col = db.code
            col.insert_one(data)
        except DuplicateKeyError:
            client.close()
            print('closed db connection')
            to_json(data, code)
            print('Wrote to json')
    else:
        print('data type is not dict')
        client.close()
        print('closed db connection')
    return

def write_list_to_file(zip_list, filename):
    """ Write the zip codes to csv file.
        
        :param zip_list: the list created from the zip codes dataframe
        :type zip_list: list stings
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "w") as z_list:
        z_list.write(",".join(str(z) for z in zip_list))
    return
        
def read_list_from_file(filename):
    """ Read the zip codes list from the csv file.
        
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "r") as z_list:
        return z_list.read().strip().split(',')

def get_weather(codes):
    ''' Get the weather from the API and load it to the database. 
    
    :param codes: list of zip codes
    :type codes: list of strings
    '''
    client = check_db_access(host, port)
    for code in codes:
        data = {}
        set_location(code)
        data.update({'zipcode': code,
                     'current': current(),
                     'five_day': five_day(),
                    })
        load(data, client)
    client.close()
    return

In [4]:
codes = read_list_from_file(filename)
get_weather(codes)

Connection made
You have write access


NotFoundError: The searched item was not found.
Reason: Unable to find the resource